<a href="https://colab.research.google.com/github/kimgu11/NLP_2023/blob/main/example/maze_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import random
import time

In [2]:
class Maze:  #미로 생성 코드
    view_delay_sec = 1.0

    def __init__(self, height, width, ratio=0.5):
        self.height = height
        self.width = width
        self.start_row, self.start_col = 1, 1
        self.end_row, self.end_col = height, width

        self._maze_init()
        self._fill(ratio)

    def _maze_init(self):
        hwall = ['#'] * (self.width + 2)
        self.maze = [hwall]
        for row in range(1, self.height + 1):
            self.maze.append(['#'] + [' '] * self.width + ['#'])
        self.maze.append(hwall)

    def _fill(self, ratio):
        empty_slots = []
        for row in range(1, self.height + 1):
            for col in range(1, self.width + 1):
                if (row, col) not in [(1, 1), (self.height, self.width)]:
                    empty_slots.append((row, col))

        n_blk = int(self.height * self.width * ratio)
        inner_walls = random.choices(empty_slots, k=n_blk)
        for row, col in inner_walls:
            self.maze[row][col] = '#'

    def in_range(self, row, col):
        return 0 <= row <= self.height and 0 <= col <= self.width

    def is_empty(self, row, col):
        return self.maze[row][col] == ' '

    def is_wall(self, row, col):
        return self.in_range(row, col) and self.maze[row][col] == '#'

    def save(self, filename):
        try:
            with open(filename, "wb") as f:
                pickle.dump(self, f)
            return True
        except IOError:
            return False

    @classmethod
    def load(cls, filename):
        try:
            with open(filename, "rb") as f:
                obj = pickle.load(f)
            return obj
        except IOError:
            return None

    def __repr__(self):
        repr = []
        for row in self.maze:
            repr.append(''.join(row))
        return '\n'.join(repr)

    def is_valid_path(self, path):
        def _neighbors(r1, c1, r2, c2):
            return (r1 - r2, c1 - c2) in [(1, 0), (0, 1), (-1, 0), (0, -1)]

        if len(path) < 1 or path[0] != (1, 1):
            return False

        for i in range(1, len(path)):
            if not self.is_empty(*path[i]):
                return False

            if not _neighbors(*path[i - 1], *path[i]):
                return False

        if path[-1] != (self.height, self.width):
            return False

        return True

    def view_path(self, path):
        if not self.is_valid_path(path):
            print("view_path(): Not a valid path")
            return

        print("view_path() starts from (1, 1)... ")
        print()
        time.sleep(Maze.view_delay_sec)

        cnt = 0
        for row, col in path:
            cnt += 1
            print('current =', (row, col), ', length =', cnt)
            self.maze[row][col] = '@'
            print(self)
            print()
            time.sleep(Maze.view_delay_sec)

        print("view_path() finished at ({}, {}), length = {}".format(self.height, self.width, len(path)))
        print()

        for row, col in path:
            self.maze[row][col] = ' '

In [3]:
if __name__ == '__main__':
    # 미로 1개 생성. H x W
    # ratio는 (H x W) 안의 칸 중에서 벽 '#'의 비율
    HEIGHT, WIDTH, RATIO = 6, 8, 0.3
    maze = Maze(HEIGHT, WIDTH, RATIO)

    # 화면 출력
    print(maze)

    # 미로 1개를 파일로 저장
    save_result = maze.save("sample.maze")
    if not save_result:
        print("save() failed")
        exit(0)

    # 미로 1개를 파일에서 불러옮
    # classmethod 이므로 변수가 아니라 클래스 이름(Maze)으로 호출
    # 리턴값은 Maze 클래스의 인스턴스
    maze = Maze.load("sample.maze")
    if not maze:
        print("load() failed")
        exit(0)

    # (1, 1) 로 시작하지 않거나,
    # (HEIGHT, WIDTH) 로 끝나지 않거나,
    # 이동 방향이 상하좌우 네 방향이 아니거나
    # 벽'#'이 포함된 경로는 invalid
    path1 = [(1, 1), (3, 3)]
    path2 = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
    print('path1 =', path1)
    print('path1 valid? =', maze.is_valid_path(path1))
    print()
    print('path2 =', path2)
    print('path2 valid? =', maze.is_valid_path(path2))

    # invalid path는 view_path()를 호출해도 그려서 보여주지 않음
    maze.view_path(path1)
    print()

##########
# #   #  #
#    #   #
#  #  #  #
#   ##  ##
# #      #
# ### #  #
##########
path1 = [(1, 1), (3, 3)]
path1 valid? = False

path2 = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
path2 valid? = False
view_path(): Not a valid path



In [4]:
def shortest_path(maze):
    maze_width = maze.width+2 #maze의 너비 (벽 포함)
    maze_height = maze.height+2 #maze의 높이 (벽 포함)
    points = [(1,1)] # 방문할 points
    path_count = [[-1]*maze_width for _ in range(maze_height)] # path_count[x][y]는 (1,1)에서 (x,y)로 가는 최단거리를 의미
                                                               # 해당 리스트를 채워 나가며 최단 경로를 구함
    path = [[-1]*maze_width for _ in range(maze_height)] # 최단 경로를 저장할 list (list에는 (x,y) 형식의 tuple이 들어감)
                                                         # path[x][y]=(a,b) 라면 (a,b)에서 (x,y)로 갔음을 의미

    path_count[1][1] = 1 # (1,1)에서 출발하므로 path_count[1][1]은 1
    while len(points) > 0: # 더이상 방문할 지점이 없을때까지 방문
        point = points.pop(0) # 방문할 지점을 가져옴
        x = point[0]
        y = point[1]
        if x-1 > 0 and maze.maze[x-1][y] != '#' and path_count[x-1][y] == -1: # (x-1,y)이 벽이 아니며, 지나갈수 있고, 이미 방문한곳이 아니라면
            points.append((x-1, y)) # 방문할 지점에 추가
            path_count[x-1][y] = path_count[x][y] +1 # 현재 지점에서 1칸만 더가면 되므로 거리에 1을 더해서 넣어줌
            path[x-1][y] = (x,y) # 패스를 저장
        if x+1 < maze_height-1 and maze.maze[x+1][y] != '#' and path_count[x+1][y] == -1: # (x+1,y)이 벽이 아니며, 지나갈수 있고, 이미 방문한곳이 아니라면
            points.append((x+1, y))
            path_count[x+1][y] = path_count[x][y] +1
            path[x+1][y] = (x,y)
        if y-1 > 0 and maze.maze[x][y-1] != '#' and path_count[x][y-1] == -1: # (x,y-1)이 벽이 아니며, 지나갈수 있고, 이미 방문한곳이 아니라면
            points.append((x, y-1))
            path_count[x][y-1] = path_count[x][y] +1
            path[x][y-1] = (x,y)
        if y+1 < maze_width-1 and maze.maze[x][y+1] != '#' and path_count[x][y+1] == -1: # (x,y+1)이 벽이 아니며, 지나갈수 있고, 이미 방문한곳이 아니라면
            points.append((x, y+1))
            path_count[x][y+1] = path_count[x][y] +1
            path[x][y+1] = (x,y)

    # 최단 경로를 추출하는 부분
    x = maze_height - 2  # 목적지에서 부터 경로를 추출하므로 x좌표는 maze_height-2
    y = maze_width - 2 # 목적지에서 부터 경로를 추출하므로 y좌표는 maze_width-2
    path_list = []
    if path_count[maze_height-2][maze_width-2] != -1: # -1은 도달할수 없음을 의미
        while True:
            path_list.insert(0, (x,y)) # 경로에 좌표를 추가
            if (x == 1 and y == 1): # (1,1)에서 출발해서 목적지에 도달하는것이므로 (1,1)까지 진행
                break
            next_point = path[x][y]  # 다음 좌표를 얻어옴
            x = next_point[0] # next_point는 (x,y) 형식의 tuple이므로 x좌표를 구함
            y = next_point[1] # next_point는 (x,y) 형식의 tuple이므로 y좌표를 구함


    #print(str(path_count[maze_height-2][maze_width-2]))
    #print(path_list)

    return path_list

In [5]:
maze_sample = Maze(height=10, width=8, ratio=0.3)
print(maze_sample)
print()

mypath = shortest_path(maze_sample)
print('mypath =', mypath)
print('length =', len(mypath))

if mypath:
    print('valid? =', maze_sample.is_valid_path(mypath))
    print()
    input('press enter to continue...')
    maze_sample.view_path(mypath)

##########
#        #
##   #   #
####     #
# ## #   #
#  ##  ###
# #      #
##       #
#      # #
####     #
#   #    #
##########

mypath = [(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6), (8, 6), (9, 6), (10, 6), (10, 7), (10, 8)]
length = 17
valid? = True

press enter to continue...
view_path() starts from (1, 1)... 

current = (1, 1) , length = 1
##########
#@       #
##   #   #
####     #
# ## #   #
#  ##  ###
# #      #
##       #
#      # #
####     #
#   #    #
##########

current = (1, 2) , length = 2
##########
#@@      #
##   #   #
####     #
# ## #   #
#  ##  ###
# #      #
##       #
#      # #
####     #
#   #    #
##########

current = (2, 2) , length = 3
##########
#@@      #
##@  #   #
####     #
# ## #   #
#  ##  ###
# #      #
##       #
#      # #
####     #
#   #    #
##########

current = (2, 3) , length = 4
##########
#@@      #
##@@ #   #
####     #
# ## #   #
#  ##  ###
# #      #
##       #
#      # #
####     #